# Custom MLP Model for Flower Classification
I created a Custom MultiLayer Perceptron (MLP) model using tensorflow Keras to classify images of flowers. I utilized data augmentation and ImageDataGenerator to preprocess the images, followed by training a custom MLP model.


## To Run in Google Colab

In [2]:
import zipfile
import os

from google.colab import drive
drive.mount('/content/drive')

# List MyDrive before t
!ls /content/drive/MyDrive/



# Open dataset from zipfile
zip_path = "/content/drive/MyDrive/flowers.zip"
extract_path = '/content/drive/MyDrive/'

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset unzipped successfully!")



# lists all files in Google Drive
!ls /content/drive/MyDrive/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 CNN_AutoEncoder.ipynb	        flowers_train_validation		    VAE.ipynb
 CNNS_Transfer_Learning.ipynb   flowers.zip				    WGAN.ipynb
 Custom_MLP_Model.ipynb        'MultiLayer_Perceptron_(MLP) -Model.ipynb'
Dataset unzipped successfully!
 CNN_AutoEncoder.ipynb	        flowers_train_validation		    VAE.ipynb
 CNNS_Transfer_Learning.ipynb   flowers.zip				    WGAN.ipynb
 Custom_MLP_Model.ipynb        'MultiLayer_Perceptron_(MLP) -Model.ipynb'


## Run on Local Machine

In [3]:
'''

import os
import zipfile

# Define the paths for the zip file and extraction directory
zip_path = r"C:\path\to\your\flowers.zip"  # Update this to your zip file's actual path
extract_path = r"C:\path\to\extract\directory"  # Update this to your desired extraction directory

# Ensure the extraction path exists
os.makedirs(extract_path, exist_ok=True)

# Extract the zip file
try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("Dataset unzipped successfully!")
except FileNotFoundError:
    print(f"Error: The file {zip_path} does not exist.")
except zipfile.BadZipFile:
    print(f"Error: The file {zip_path} is not a valid zip file.")

'''


'\n\nimport os\nimport zipfile\n\n# Define the paths for the zip file and extraction directory\nzip_path = r"C:\\path\to\\your\x0clowers.zip"  # Update this to your zip file\'s actual path\nextract_path = r"C:\\path\to\\extract\\directory"  # Update this to your desired extraction directory\n\n# Ensure the extraction path exists\nos.makedirs(extract_path, exist_ok=True)\n\n# Extract the zip file\ntry:\n    with zipfile.ZipFile(zip_path, \'r\') as zip_ref:\n        zip_ref.extractall(extract_path)\n    print("Dataset unzipped successfully!")\nexcept FileNotFoundError:\n    print(f"Error: The file {zip_path} does not exist.")\nexcept zipfile.BadZipFile:\n    print(f"Error: The file {zip_path} is not a valid zip file.")\n\n'

In [4]:
# Google CoLab
train_dir  = '/content/drive/MyDrive/flowers_train_validation/train'
validation_dir = '/content/drive/MyDrive/flowers_train_validation/validation'

# Local Machine
#Train = 'C:/path to local drive for extracted files /flowers_train_validation/train'
#Validation = 'C:/path to local drive for extracted files /flowers_train_validation/validation'
TARGET_SIZE=(150,150)

In [5]:
# Import TensorFlow and Keras
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tqdm import tqdm
from tensorflow.keras import layers, models

## 1: Data Preparation and Augmentation
We'll use the ImageDataGenerator class to augment our training data and rescale the images. Data augmentation helps in increasing the diversity of the training data, which helps in reducing overfitting.


In [6]:
# Create ImageDataGenerators for training data
train_datagen = ImageDataGenerator(rescale=1./255,
                                    rotation_range=20,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    fill_mode='nearest')

# create validation generator with rescale, no augmentation
validation_datagen = ImageDataGenerator(rescale=1./255)

In [7]:
train_generator = train_datagen.flow_from_directory(
    train_dir,  # This is the target directory
    target_size=(150, 150),  # All images will be resized to 150x150
    batch_size=128,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

Found 3177 images belonging to 5 classes.
Found 865 images belonging to 5 classes.


## 2: Building the Custom MLP Model

### 2.1 Custom Categorical Crossentropy Loss

**Defining Custom Categorical CrossEntropy Loss**

The categorical cross-entropy formula is:
$$L = - \sum_{i=1}^{N} y_i \log(\hat{y_i})$$
Where:
- $N$ is the number of classes.
- $y_i$ is the true label (one-hot encoded, 1 for the correct class, and 0 for the others).
- $\hat{y_i}$ is the predicted probability for the corresponding class (output of a `softmax`).

For each sample
- Multiply with One-Hot Labels: Multiply the logarithm of the predictions with the corresponding one-hot encoded labels (y_true), so only the correct class’s prediction contributes to the loss.
- Sum the Results: Sum the result of the above operation across all classes for each sample.


In [8]:
# Custom categorical entropy loss function
def my_categorical_crossentropy(y_true, y_pred):

    # Clip prediction values to avoid log(0) error
    y_pred = tf.clip_by_value(y_pred, 1e-10, 1.0)

    # Compute the categorical cross-entropy loss
    loss = -tf.reduce_sum(y_true * tf.math.log(y_pred), axis=-1)

    # Return the mean loss over the batch
    loss = tf.reduce_mean(loss)

    return loss


In [9]:
# Define y_true (one-hot encoded labels) and y_pred (predicted probabilities)
y_true = np.array([
    [1, 0, 0, 0, 0],  # Class 0
    [0, 1, 0, 0, 0],  # Class 1
    [0, 0, 0, 1, 0]   # Class 3
])

y_pred = np.array([
    [0.8, 0.0, 0.1, 0.05, 0.05],  # Model is confident about class 0
    [0.1, 0.6, 0.1, 0.1, 0.1],    # Model is confident about class 1
    [0.05, 0.05, 0.05, 0.8, 0.05]  # Model is confident about class 3
])

# Convert y_true and y_pred to tensors
y_true_tensor = tf.convert_to_tensor(y_true, dtype=tf.float32)
y_pred_tensor = tf.convert_to_tensor(y_pred, dtype=tf.float32)

# Compute the custom loss
loss = my_categorical_crossentropy(y_true_tensor, y_pred_tensor)

# Run the computation in a TensorFlow session
print("Custom categorical entropy loss: ", loss)

Custom categorical entropy loss:  tf.Tensor(0.31903753, shape=(), dtype=float32)


### 2.2 Custom Layers

### 2.2.1 MyFlatten Layer

In [10]:
# Custom Flatten layer
class MyFlatten(tf.keras.layers.Layer):
    def __init__(self):
        super(MyFlatten, self).__init__()

    def call(self, inputs):
        # Flatten the input
        return tf.reshape(inputs, [inputs.shape[0], -1])

    def compute_output_shape(self, input_shape):
        # The output shape is (batch_size, flattened_dims)
        # Calculate the product of all dimensions except the batch size (input_shape[0])
        flatten_dim = 1
        for dim in input_shape[1:]:
            if dim is not None:
                flatten_dim *= dim
            else:
                # If any dimension is None, return None (because we cannot calculate the product statically)
                return (input_shape[0], None)

        return (input_shape[0], flatten_dim)


In [11]:
# Example of input with size (3, 3)
input_data = tf.constant([[1, 2, 3], [4, 5, 6], [7, 8, 9]], dtype=tf.float32)

# Reshape input_data to add batch dimension (batch_size, 3, 3)
input_data = tf.expand_dims(input_data, axis=0)  # Adding batch size of 1, so shape is (1, 3, 3)

# Instantiate and apply the custom flatten layer
flatten_layer = MyFlatten()
flattened_output = flatten_layer(input_data)

# Print the result
print("Input shape:", input_data.shape)
print("Flattened output:", flattened_output.numpy())
print("Flattened output shape:", flattened_output.shape)

Input shape: (1, 3, 3)
Flattened output: [[1. 2. 3. 4. 5. 6. 7. 8. 9.]]
Flattened output shape: (1, 9)


### 2.2.2 MyDense Layer

**Defining Custom Dense Layer supporting activation**:

- **init()**: initialize units and activations
- **build()**: initialize weights and biases
- **call()**: forward pass
- **compute_output_shape()**: define output shape of the layer. It is always (batch_size, units)

In [12]:
class MyDense(tf.keras.layers.Layer):

    def __init__(self, units=32, activation=None):
        super(MyDense, self).__init__()
        self.units = units
        self.activation = tf.keras.activations.get(activation)


    def build(self, input_shape):

        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True)
        self.b = self.add_weight(shape=(self.units,),
                                 initializer='zeros',
                                 trainable=True)


    def call(self, inputs):
        return self.activation(tf.matmul(inputs, self.w) + self.b)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.units)



### 2.3 Custom Model using Subclassing

**Defining Custom Model for Flower Prediction**:

- **init()**: create instances of layers
- **call()**: forward pass


In [13]:
# Custom model class
class MyFlowerModel(tf.keras.models.Model):
    def __init__(self, num_classes):
        super(MyFlowerModel, self).__init__()
        self.flatten = MyFlatten()
        self.dense1 = MyDense(200, activation='relu')
        self.dense2 = MyDense(150, activation='relu')
        self.dense3 = MyDense(num_classes, activation='softmax')


    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.dense1(x)
        x= self.dense2(x)
        return self.dense3(x)



In [14]:
model = MyFlowerModel(5)

## 3. Custom Training Loop

### 3.1 Create instances for Optimizer and Loss

**Create instances for optimizer and loss**

- `adam` optimizer and
- custom categorical crossentropy loss


In [15]:
optimizer = tf.keras.optimizers.Adam()
loss_object = my_categorical_crossentropy



### 3.2 Define Metrics

**Create instances for metrics (both train and validation)**

- Using `CategoricalAccuracy`defined in `tf.keras.metrics`


In [16]:
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
val_acc_metric = tf.keras.metrics.CategoricalAccuracy()



### 3.3 Custom Training Loop

The core of training is using the model to calculate the logits on specific set of inputs and compute loss (in this case **categorical crossentropy**) by comparing the predicted outputs to the true outputs. Update the trainable weights using the optimizer algorithm chosen. Optimizer algorithm requires the computed loss and partial derivatives of loss with respect to each of the trainable weights to make updates to the same.

#### 3.3.1. Gradient Calculation

**Apply gradients on optimizer**

- *optimizer*: your optimizer used to optimize the model paramenters
- *model*: your custom flower model
- *x*: input training x
- *y*: input training y

The function will use tensorflow's gradientTape to calculate the gradients and then optimize the parameters through optimizer. The function will return logits (model's predicted values) and loss_value (calculated by the loss function).



In [17]:
def apply_gradient(optimizer, model, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss_value = loss_object(y_true=y, y_pred=logits)
    grads = tape.gradient(loss_value, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return logits, loss_value




### 3.3.2 Define a Training for Each Epoch

**Task 7: train_data_for_one_epoch()**

This function performs training during one epoch. You run through all batches of training data in each epoch to make updates to trainable weights using the previous function. It will call update_state on your metrics to accumulate the value of the metrics.
Display a progress bar to indicate completion of training in each epoch (use **tqdm** for displaying the progress bar).

In [18]:
def train_data_for_one_epoch():
    losses = []
    steps = train_generator.samples // train_generator.batch_size
    pbar = tqdm(total=steps, position=0, leave=True, bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} ')

    for step, (x_batch_train, y_batch_train) in enumerate(train_generator):
        if step >= steps - 1:
            break

        # Call the function to apply gradient and get logits and loss
        logits, loss_value = apply_gradient(optimizer, model, x_batch_train, y_batch_train)

        losses.append(loss_value)

        # Update state
        train_acc_metric(y_batch_train, logits)

        # Update progress bar
        pbar.set_description("Training loss for step %s: %.4f" % (int(step), float(loss_value)))
        pbar.update()

    return losses



#### 3.3.3 Perform Validation

**perform_validation()**

- Change **enumerate(test)** to  **enumerate(validation_generator)**, due to the fact we use **imageDataGenerator** not **tfds**
- Add **STEPS= validation_generator.samples // validation_generator.batch_size** before the loop (needed for stopping the generator)
- Stop the loop after reaching the number of STEPS. i.e. add the statements at the end in the loop: **if step >= STEPs - 1: break**.  (*Note*: same logic as above.)

In [19]:
def perform_validation():
    losses = []

    # Calculate the number of steps
    STEPS = validation_generator.samples // validation_generator.batch_size

    for step, (x_val, y_val) in enumerate(validation_generator):
        val_logits = model(x_val)
        val_loss = loss_object(y_true=y_val, y_pred=val_logits)
        losses.append(val_loss)

        # Update metrics
        val_acc_metric(y_val, val_logits)

        # Stop after reaching the number of STEPS
        if step >= STEPS - 1:
            break

    return losses



### 3.3.4 Model fit

**Perform model fit using training Loops**

1. Perform training over all batches of training data.
2. Get values of metrics.
3. Perform validation to calculate loss and update validation metrics on test data.
4. Reset the metrics at the end of epoch.
5. Display statistics at the end of each epoch.


In [20]:
# your model fitting
epochs = 10
epochs_val_losses, epochs_train_losses = [], []
for epoch in range(epochs):

    losses_train = train_data_for_one_epoch()
    train_acc = train_acc_metric.result()

    losses_val = perform_validation()
    val_acc = val_acc_metric.result()

    losses_train_mean = np.mean(losses_train)
    losses_val_mean = np.mean(losses_val)
    epochs_val_losses.append(losses_val_mean)
    epochs_train_losses.append(losses_train_mean)

    print('\n Epoch %s: Train loss: %.4f  Validation Loss: %.4f, Train Accuracy: %.4f, Validation Accuracy %.4f' % (epoch + 1, float(losses_train_mean), float(losses_val_mean), float(train_acc), float(val_acc)))

    train_acc_metric.reset_state()
    val_acc_metric.reset_state()


Training loss for step 22: 11.1043:  96%|█████████▌| 23/24 



 Epoch 1: Train loss: 11.5035  Validation Loss: 10.1205, Train Accuracy: 0.2303, Validation Accuracy 0.2812


Training loss for step 22: 4.2654:  96%|█████████▌| 23/24 



 Epoch 2: Train loss: 8.5405  Validation Loss: 3.3775, Train Accuracy: 0.2711, Validation Accuracy 0.3517


Training loss for step 22: 1.5262:  96%|█████████▌| 23/24 


...


 Epoch 8: Train loss: 1.3132  Validation Loss: 1.2608, Train Accuracy: 0.4296, Validation Accuracy 0.4286


Training loss for step 22: 1.3031:  96%|█████████▌| 23/24 



 Epoch 9: Train loss: 1.2888  Validation Loss: 1.2673, Train Accuracy: 0.4608, Validation Accuracy 0.4610


Training loss for step 22: 1.1899:  96%|█████████▌| 23/24 



 Epoch 10: Train loss: 1.2790  Validation Loss: 1.3452, Train Accuracy: 0.4399, Validation Accuracy 0.4298


**Model Summary**
- Layer types (MyFlatten, not Keras.Flatten, MyDense, not keras.Dense)
- Output shapes (batch size, units)
- Parameters (based on fully connected neurons)

In [21]:
# your model summary
model.summary()


Model: "my_flower_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 my_flatten_1 (MyFlatten)    multiple                  0         
                                                                 
 my_dense (MyDense)          multiple                  13500200  
                                                                 
 my_dense_1 (MyDense)        multiple                  30150     
                                                                 
 my_dense_2 (MyDense)        multiple                  755       
                                                                 
Total params: 13531105 (51.62 MB)
Trainable params: 13531105 (51.62 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
